# Ejemplo de uso

In [6]:
import mib_v2_3_2 as mb
import tpproc as tp
from collections import Counter  #regresa un diccionario con conteos
import glob
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from collections import OrderedDict #diccionarios ordenados
import numpy as np
from itertools import product

[nltk_data] Downloading package punkt to /home/roberto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Modelo de un problema

$P(ABC) = P(A)P(B|A)P(C|AB)$.

### Probabilidades

<table width="90%">
    <tr>
        <td valign="top">
            <table width="50%">
              <tr>
                <th> $A$</th>
                <th><p style="text-align:center;"> $P(A)$</p></th>
              </tr>
              <tr>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
              </tr>
            </table>
        </td>
        <td valign="top">
            <table width="60%">
              <tr>
                <td style="height:1px;"> &nbsp;</td>
                <td colspan="2"><p style="text-align:center;">$P$($B$|$A$)</p></td>
              </tr>
              <tr>
                <td>$A$</td>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
                <td><p style="text-align:center;">$0.8$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
              </tr>
            </table>        
        </td>
        <td valign="top">
            <table width="90%">
              <tr>
                <td style="height:1px;"> &nbsp;</td>
                <td colspan="2"><p style="text-align:center;">$P$($C$|$AB$)</p></td>
              </tr>
              <tr>
                <td>$AB$</td>
                <td><p style="text-align:center;">$0$</p></td>
                <td><p style="text-align:center;">$1$</p></td>
                <td><p style="text-align:center;">$2$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$00$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
                <td><p style="text-align:center;">$0.8$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$01$</p></td>
                <td><p style="text-align:center;">$0.3$</p></td>
                <td><p style="text-align:center;">$0.5$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$10$</p></td>
                <td><p style="text-align:center;">$0.4$</p></td>
                <td><p style="text-align:center;">$0.5$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
              </tr>
              <tr>
                <td><p style="text-align:center;">$11$</p></td>
                <td><p style="text-align:center;">$0.1$</p></td>
                <td><p style="text-align:center;">$0.7$</p></td>
                <td><p style="text-align:center;">$0.2$</p></td>
              </tr>
            </table>        
        </td>
    </tr>
</table>

## Creación de datos

### Declaración de diccionarios

In [2]:
# Tablas
# {(A,): P(A)}
dA = {
    (0,): 0.3,
    (1,): 0.7
}

# {(variables independientes): {variables: p}}

# {(A,):{(B,): P(B|A)}}
dB_A = {
    (0,):{(0,): 0.2, (1,): 0.8},
    (1,):{(0,): 0.3, (1,): 0.7}
}

# {(A,B):{(C,): P(C|A,B)}}
dC_AB = {
    (0,0): {(0,): 0.1, (1,): 0.8, (2,): 0.1},
    (0,1): {(0,): 0.3, (1,): 0.5, (2,): 0.2},
    (1,0): {(0,): 0.4, (1,): 0.5, (2,): 0.1},
    (1,1): {(0,): 0.1, (1,): 0.7, (2,): 0.2}
    }


### Declaración de variables

In [3]:
A = mb.Var(name='A', values=set([0,1]))
B = mb.Var(name='B', values=set([0,1]))
C = mb.Var(name='C', values=set([0,1,2]))

### Declaración de distribuciones 

In [4]:
# P(A)
PA = mb.Distrib(table=dA, vars=(A,))
# P(B|A)
PB_A = mb.Distrib(table=dB_A, vars=(B,), parents=(A,))
# P(C|AB)
PC_AB = mb.Distrib(table=dC_AB, vars=(C,), parents=(A, B))

## Creación de la pregunta

In [5]:
# Probabilidad conjunta (Especificación)
PABC = mb.Specification(set([A,B,C]), (PA, PB_A, PC_AB))
# Generador de consultas (Pregunta)
QABC = mb.Question(PABC)

## Creación de consultas

### Distribuciones

#### De una variable

In [6]:
PB = QABC.DistributionQuery(set([B]))
PB.table

{(0,): 0.27, (1,): 0.7299999999999999}

In [7]:
PB = QABC.DistributionQuery(set([B]), aproximation=True)
PB.table

{(0,): 0.2748, (1,): 0.7322}

#### Conjuntas

In [8]:
PAB = QABC.DistributionQuery(set([A,B]))
PAB.table

{(0, 0): 0.06, (0, 1): 0.24, (1, 0): 0.21, (1, 1): 0.48999999999999994}

In [9]:
PAB = QABC.DistributionQuery(set([A,B]), aproximation=True)
PAB.table

{(0, 0): 0.063, (0, 1): 0.241, (1, 0): 0.2094, (1, 1): 0.4881}

#### Condicional

In [10]:
PA_B = QABC.DistributionQuery(set([A]), set([B]))
parents_names = tuple([v.name for v in PA_B.parents])
vars_names = tuple([v.name for v in PA_B.vars])
print(parents_names, vars_names)
PA_B.table

('B',) ('A',)


{(0,): {(0,): 0.2222222222222222, (1,): 0.7777777777777777},
 (1,): {(0,): 0.32876712328767127, (1,): 0.6712328767123288}}

In [11]:
PA_B = QABC.DistributionQuery(set([A]), set([B]), aproximation=True)
parents_names = tuple([v.name for v in PA_B.parents])
vars_names = tuple([v.name for v in PA_B.vars])
print(parents_names, vars_names)
PA_B.table

('B',) ('A',)


{(0,): {(0,): 0, (1,): 0}, (1,): {(0,): 0, (1,): 0}}

### Consulta de probabilidades

In [12]:
# P(A=0|C=0)
p = QABC.Query((A,), (C,), (0,), (0,))
p

0.3696682464454977

In [13]:
# P(A=0,B=0)
p = QABC.Query(vars=(A,B), vars_values=(0,0))
p

0.06

### Consultas de valores más probables

In [14]:
#P(A|B=0)
p = QABC.Query(vars=(A,), indep=(B,), indep_values=(0,))
p

(('A',), (1,), 0.7777777777777777)

In [15]:
#P(A=0|B)
p = QABC.Query(vars=(A,), indep=(B,), vars_values=(0,))
p

(('B',), (1,), 0.32876712328767127)

In [16]:
#P(AB)
p = QABC.Query(vars=(A,B))
p

(('A', 'B'), (1, 1), 0.48999999999999994)

# Problema de autoria de textos 

## Planteamiento

### Especificación

#### Variables

- $A$ : Variable para los autores
- $T$ : Variable para los tipos de textos
- $W_I$ : Variable para las palabras

##### Variable $A$

$|A| = 10 $

##### Variable $T$

$ T = \{'minicuento', 'fábula', 'cuento'\} $

##### Variable $W_i$

$ W_i = \{0,1\}$, donde cada $W_i$ representa una variable para cada palabra.

Si $W_i$ es 0, entonces la palabra no esta. Si $W_i$ es 1 entonces la palabra esta.

#### Modelo de descomposición

$P(A)P(T|A)\displaystyle \prod_{i=0}^{n} P(W_i|AT)$

## Implementación

### Obtención de datos para el test

In [2]:
archivos = glob.glob('./Train/*/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_train = tp.lee_cuentos(archivos)

archivos = glob.glob('./Test/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_test = tp.lee_cuentos(archivos,test=True)

leyendo...
tamaño del contenido de archivos cargados:             1 KB
leyendo...
tamaño del contenido de archivos cargados:             0 KB


#### Data Frames de trabajo

In [3]:
s = df_train.texto.str.cat(sep=' ').split()
print(len(s))
conteos=Counter(s)
nuevo_vocabulario = list(set([palabra for palabra in s if conteos[palabra] < 10 and conteos[palabra] > 5]))
print(len(nuevo_vocabulario))

7210
104


In [4]:
df_train_T = df_train.copy()
df_train_T['nuevo_texto'] = df_train_T.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in nuevo_vocabulario]).\
    apply(lambda x : ' '.join(x))
df_train_T['Conteos']=df_train_T.nuevo_texto.str.split().apply(Counter)


df_train_T.head()

,titulo,tipo,autor,texto,nuevo_texto,Conteos
0,la sombra,minicuento,gibrán jalil gibrán,cierto día junio hierba dijo sombra olmo mueve...,cierto respondió cielo árbol viento sol mirada...,"{'cierto': 1, 'respondió': 1, 'cielo': 1, 'árb..."
1,la búsqueda,cuento,gibrán jalil gibrán,hace mil años dos filósofos encontraron cuesta...,encontraron dónde fuente respondió fuente sol ...,"{'encontraron': 1, 'dónde': 1, 'fuente': 3, 'r..."
2,dos poemas,cuento,gibrán jalil gibrán,varios siglos atrás camino atenas encontraron ...,atrás encontraron cómo respondió escrito aquel...,"{'atrás': 1, 'encontraron': 1, 'cómo': 1, 'res..."
3,"aquel viejo, viejo vino",cuento,gibrán jalil gibrán,vez hombre rico orgulloso bodega vino allí vas...,pensar,{'pensar': 1}
4,tres regalos,cuento,gibrán jalil gibrán,cierta vez ciudad becharre vivía amable prínci...,ciudad hecho cómo respondió piel aquel silencio,"{'ciudad': 1, 'hecho': 1, 'cómo': 1, 'respondi..."


In [5]:
df_test_T = df_test.copy()
df_test_T['nuevo_texto']=df_test_T.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in nuevo_vocabulario]).\
    apply(lambda x : ' '.join(x))
df_test_T['nuevo_total']=df_test_T.nuevo_texto.str.split().apply(len)
df_test_T['Conteos']=df_test_T.nuevo_texto.str.split().apply(Counter)


df_test_T.head()

,titulo,texto,nuevo_texto,nuevo_total,Conteos
0,dos seres iguales,cierto día profeta sharía encontró niña jardín...,cierto jardín señor respondió aquel miró respo...,16,"{'cierto': 1, 'jardín': 2, 'señor': 1, 'respon..."
1,el león y el pastor,yendo león montaña erró camino pasando lugar l...,lugar vio sino lado lugar aquel gran lado hech...,10,"{'lugar': 2, 'vio': 1, 'sino': 1, 'lado': 2, '..."
2,el caballo y el lobo,estación blandos céfiros hacen verdear campos ...,cierto todas aquel señor aquel parte,6,"{'cierto': 1, 'todas': 1, 'aquel': 2, 'señor':..."
3,prisa,pesar torpor ojos hinchados aire recién salido...,aire todas dónde sino parte lugar fuente cómo ...,11,"{'aire': 1, 'todas': 1, 'dónde': 2, 'sino': 1,..."
4,la melancolía del viajero,veces vuelven largo viaje conservan toda vida ...,veces largo todas gran cómo todavía casi tener...,13,"{'veces': 1, 'largo': 1, 'todas': 1, 'gran': 1..."


In [ ]:
voc_dict = OrderedDict(zip(nuevo_vocabulario,range(len(nuevo_vocabulario))))
v = list(voc_dict.keys())

#### Conteos

In [ ]:
# Ocurrencias de cada autor y conteos de ocurrencias y de número de autores
oc_autor = Counter(df_train_T.autor)
tot_oc_aut = np.sum(list(oc_autor.values()))
# Ocurrencias de cada tipo y conteos de ocurrencias y de número de tipos
oc_tipo = Counter(df_train_T.tipo)
tot_oc_tipo = np.sum(list(oc_tipo.values()))

### Variables

In [ ]:
# Autor
autor_val = dict(zip(oc_autor.keys(),range(len(oc_autor.keys()))))
val_autor = dict(zip(range(len(oc_autor.keys())),oc_autor.keys()))
A = mb.Var('A',set(oc_autor.keys()))
# Tipos
tipo_val = dict(zip(oc_tipo.keys(),range(len(oc_tipo.keys()))))
val_tipo = dict(zip(range(len(oc_tipo.keys())),oc_tipo.keys()))
T = mb.Var('T',set(oc_tipo.keys()))

# Palabras
W = {}
for w in voc_dict:    # vocabulario reducido
    W[w] = mb.Var(w,set([0,1]))

### Identificación de parámetros (estadísticas)

#### Método:

1. Descomposición exacta: $P(A)P(T|A)P(W|AT)$
2. Calcular las estadísticas por cada distribución de la descomposición
3. Para la variable Autor $P(A)$:
    - 3.1 Contar el número total de ocurrencias de autores en Train: $n_a$.
    - 3.2 Contar el número de ocurrencias de cada autor en Train: $n^i$.
    - 3.3 Dividir este número entre el número total de ocurrencias de autores en Train: $\displaystyle \frac{n^i}{n_a}$.
4. Para la variable Tipo $P(T|A)$:
    - 4.1 Obtener el conjunto de autores (valores únicos) $a$; entradas a la Tabla.
    - 4.2 Obtener el conjunto de tipos (valores únicos) $j$; estas son las salidas de la Tabla.
    - 4.3 Calcular el producto cartesiano de las combinaciones posibles (a,j).
    - 4.4 Contar el número de ocurrencias totales de cada $j$ por cada autor: $n^j_a$.
    - 4.5 Dividir este número entre el número total de tipos por autor: $\displaystyle \frac{n^j_a}{\sum n^j_a}$.
    - 4.6 Aplicar la corrección de Laplace para evitar probabilidades en $0$.
5. Para las variables W (palabras) $P(W|AT)$: **NOTA** se trata de calcular N Tablas, donde N es el tamaño del vocabulario.
    - 5.1 Calcular el producto cartesiano de las combinaciones posibles (a,j); estas son las entradas a cada Tabla.
    - 5.2 Por cada combinación, calcular la probabilidad de cada palabra del vocabulario, en función de su ocurrencia en esa combinación, aplicando la corrección de Laplace. 

#### Distribución P(A)

In [ ]:
n_= []
for autor in oc_autor:
    n_.append(((autor,),oc_autor[autor]/tot_oc_aut))

#Dicionario de valores de probabilidad
dA = dict(n_)
#Distribución de probabilidad
PA = mb.Distrib(table = dA, vars=(A,))
print(A.getValues())
PA.table

#### Distribución P(T|A)

In [ ]:
# Ocurrencias (conteos) de cada combinación (tipo,autor)
conteo_pares = Counter(zip(df_train_T.autor,df_train_T.tipo))


# Combinaciones (tipo,autor)
autores = list(set(df_train_T.autor))
tipos = list(set(df_train_T.tipo))

dT_A = {}

for autor, tipo in product(*[autores,tipos]):
    par = (autor, tipo)
    ak = (autor,)
    
    if ak in dT_A.keys():
        if par in conteo_pares.keys():
            dT_A[ak][(tipo,)] = conteo_pares[par]
        else:
            dT_A[ak][(tipo,)] = 0 
    else:
        if par in conteo_pares.keys():
            dT_A[ak] = {(tipo,): conteo_pares[par]}
        else:
            dT_A[ak] = {(tipo,): 0}


# Corrección de Laplace en el caso general
# Se corrige la misma tabla que se senvía como argumento; no se crea una nueva.
def Laplace_gral(tabla):
    n = len(tabla[list(tabla.keys())[0]]) 
    for k in tabla.keys():
        registro = tabla[k]
        n_j = np.sum(list(registro.values()))
        for i in registro.keys():
            pb = (registro[i] + 1)/(n_j+n)
            tabla[k][i]=pb
    return

Laplace_gral(dT_A)

PT_A = mb.Distrib(dT_A, (T,), (A,))

#### $P(W|AT)$

In [ ]:
# Combinaciones (tipo,autor)
autores = list(set(df_train_T.autor))
tipos = list(set(df_train_T.tipo))
pares_ta = list(product(autores,tipos))

# Conteos de palabras en nuevo_texto por pares (a,t) en el data frame de train
conteo_w = dict(df_train_T.Conteos)
pares_train = list(zip(df_train_T.autor,df_train_T.tipo))

for i,k in enumerate(conteo_w.keys()):
    conteo_w[k] = {pares_train[i] : dict(conteo_w[k])}
    
def check(palabra,pares_ta):
    pw_ = {}
    var = W[palabra]
    
    for par in pares_ta:
        cw_1 = 0   # conteos de presencia 
        cw_0 = 0   # y ausencia en 0
        it = 0     # total de pares (a,t) contabilizados
        
        for item in list(conteo_w.values()):  # checamos cada par (a,t) en el conjunto de train
            # print(i,list(item.keys())[0], end=' ')
            it += 1
            if par in item: # checamos si el par (a,t) de entrada tiene correspondencia en Train
                if palabra in list(item.values())[0]: #checamos si la palabra existe en el par (a,t) de Train
                    cw_1 += 1
                else:
                    cw_0 += 1  #debemos contar también la no existencia por si hay igualdad
        
        if cw_0 == cw_1: # la palabra aparece por igual en todos los pares Train, o bien, no hay pares Train
            pw_1 = 0.5   #corrección de Laplace que indica que la palabra puede o no estar por igual
            pw_0 = 0.5
            
        else:
            pw_1=(1+cw_1)/(len(var.getValues())+it) #corrección de Laplace en caso de que la palabra exista
            pw_0 = 1-pw_1
        
        pw_[par] = {(0,): pw_0, (1,): pw_1}
    pw_ = OrderedDict(sorted(pw_.items()))
    return pw_
        
    
PW_AT = {}
for w in voc_dict:    # vocabulario reducido
    t = check(w,pares_ta)
    PW_AT[w] = mb.Distrib(dict(t), (W[w],), (A, T))

### Preguntas

In [ ]:
vl =  [A,T] + [W[w] for w in W] 
dl = [PA,PT_A] + [PW_AT[w] for w in PW_AT]
vars_set = set(vl)
dist_set = tuple(dl)

In [ ]:
PATW = mb.Specification(vars_set, dist_set)
QW_AT = mb.Question(PATW)

#### 1. ¿Quién escribió "El caballo y el Lobo"?

In [ ]:
dcont = df_test_T.loc[df_test['titulo'] == 'el caballo y el lobo', 'Conteos'].iloc[0]
wit = []
for w in dcont:
    if w in W.keys():
        wit.append(W[w])

**Pregunta:**
$ P(A | W_0, W_1, \dots, W_n) $, donde se busca el valor más probable de $A$.

In [ ]:
CA, VA, p = QW_AT.Query(vars=(A,), indep=tuple(wit), indep_values=tuple([1 for i in wit]), aproximation=True, N=7500)
VA